# Buscador TF-IDF no TREC-DL 2020

Aqui, é implementado um buscador com vetorização TF-IDF, que leva em conta tanto a frequência de termos em cada documento, como também a "raridade" de cada termo no corpus.

## Download do dataset

Montagem local para acesso posterior aos arquivos

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
main_path = '/content/drive/MyDrive/Unicamp-aula-2/'

import os

if not os.path.exists(main_path):
  os.makedirs(main_path)
else:
  print('Diretório já existente')

Diretório já existente


## Download de ferramentas auxiliares

Instalação do Pyserini para uso do Lucene Analyzer para fins de pré-processamento(tokenização, remoção de stopwords, stemming).

In [3]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 87.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

Download de ferramentas para avaliação do TREC DL 2020

In [ ]:
!git clone https://github.com/castorini/pyserini.git --recurse-submodules {main_path}/pyserini

In [ ]:
!cd {main_path}/pyserini/tools/eval && tar xvfz trec_eval.9.0.4.tar.gz && cd trec_eval.9.0.4 && make && cd ../../..
!cd {main_path}/pyserini/tools/eval/ndeval && make && cd ../../..

trec_eval.9.0.4/
trec_eval.9.0.4/m_prefs_pair.c
trec_eval.9.0.4/m_ndcg_p.c
trec_eval.9.0.4/m_infap.c
trec_eval.9.0.4/m_num_q.c
trec_eval.9.0.4/m_iprec_at_recall.c
trec_eval.9.0.4/form_prefs_counts.c
trec_eval.9.0.4/m_prefs_num_prefs_ful_ret.c
trec_eval.9.0.4/utility_pool.c
trec_eval.9.0.4/m_binG.c
trec_eval.9.0.4/meas_avg.c
trec_eval.9.0.4/m_gm_bpref.c
trec_eval.9.0.4/m_runid.c
trec_eval.9.0.4/m_bpref.c
trec_eval.9.0.4/m_gm_map.c
trec_eval.9.0.4/trec_eval.h
trec_eval.9.0.4/m_yaap.c
trec_eval.9.0.4/m_relstring.c
trec_eval.9.0.4/m_Rprec.c
trec_eval.9.0.4/m_prefs_avgjg.c
trec_eval.9.0.4/m_success.c
trec_eval.9.0.4/m_ndcg.c
trec_eval.9.0.4/functions.h
trec_eval.9.0.4/m_P_avgjg.c
trec_eval.9.0.4/test/
trec_eval.9.0.4/test/qrels.rel_level
trec_eval.9.0.4/test/results.test
trec_eval.9.0.4/test/qrels.test
trec_eval.9.0.4/test/out.test.qrels_jg
trec_eval.9.0.4/test/out.test.meas_params
trec_eval.9.0.4/test/out.test.a
trec_eval.9.0.4/test/out.test.prefs
trec_eval.9.0.4/test/out.test.aqcM
trec_ev

## Construção do índice invertido

In [4]:
from pyserini.analysis import Analyzer, get_lucene_analyzer

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


Código para pré-processamento textual

In [6]:

collection_path = main_path + '/collections/msmarco-passage/collection.tsv'



Foi verificado que o analisador do Lucene não elimina suficientemente stopwords.  Assim, foi utilizada adicionalmente a lista de stopwords do NLTK.

In [7]:
import nltk
import string

from nltk.stem import PorterStemmer

nltk.download('stopwords')  # Download stopwords if not already downloaded

from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words = set(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
analyzer:Analyzer = Analyzer(get_lucene_analyzer(stemmer='porter'))

def preprocess_and_tokenize(text):
  #remove previamente as stopwords do NLTK -> isso pode mais adiante interferir na contagem de termos/frquências
  tokens = text.lower().split()
  tokens = [token for token in tokens if token not in stop_words]
  text = ' '.join(tokens)
  return analyzer.analyze(text)

Em seguida, será construído o índice invertido para associar os tokens aos documentos nos quais eles ocorrem.  O uso da estrutura de dados array (sugestão do colega [Leandro Carísio](https://colab.research.google.com/drive/1hELJYqsvUyja9HPeDzc9FU8okqdIjODE?usp=sharing)) resultou em um ganho relevante em memória em relação a listas.  Tal estrutura de dados, ao contrário dos arrays do Numpy, assemelha-se a um array dinâmico, ou lista, porém ocupa bem menos memória.

Para subsidir posteriormente o algoritmo baseado em TF-IDF, é armazenada adicionalmente a frequência de termos (TF) para cada documento, na entrada correspondente.

In [11]:
import array
import pandas as pd
from collections import defaultdict
from collections import Counter
import pickle
import os

index_path = f"{main_path}/index-tf-idf.pickle"

def load_or_build_inverted_index():
  if os.path.exists(index_path):
    with open(index_path, "rb") as f:
      print("Loading index...")
      index = pickle.load(f)
  else:
    print("Building inverted index and vocabulary...")
    # set the chunk size
    chunk_size = 1000
    chunks = []
    inverted_index = dict()
    full_text = ''

    def process(row):
      tokenized_text = preprocess_and_tokenize(row[1])
      counter = Counter(tokenized_text)
      doc_length = len(tokenized_text)
      doc_id = row[0]
      for token, count in counter.items():
        if token not in stop_words:
          #Para cada token, temos 2 arrays paralelos que armazenam os documentos e as frquências dos termos
          inverted_index.setdefault(token, {"docs":array.array("L", []), "tf":array.array("f", [])})["docs"].append(int(doc_id))
          inverted_index.setdefault(token, {"docs":array.array("L", []), "tf":array.array("f", [])})["tf"].append(count/doc_length)

    chunk_id = 0
    n_documents = 0

    # iterate through the file in chunks
    for chunk in pd.read_csv(collection_path, sep='\t', header=None, chunksize=chunk_size):
      # process the chunk here
      if (chunk_id % 1000) == 0:
        print(f'Processing chunk {chunk_id}')
      for index, row in chunk.iterrows():
        process(row)
        n_documents += 1
      del(chunk)
      chunk_id += 1

    index = {"inverted_index": inverted_index, "n_documents": n_documents}

    with open(index_path, "wb") as f:
      pickle.dump(index, f)

  return index

In [12]:
index = load_or_build_inverted_index()
inverted_index = index["inverted_index"]
n_documents = index["n_documents"]

Building inverted index and vocabulary...
Processing chunk 0
Processing chunk 1000
Processing chunk 2000
Processing chunk 3000
Processing chunk 4000
Processing chunk 5000
Processing chunk 6000
Processing chunk 7000
Processing chunk 8000


KeyError: ignored

In [13]:
len(inverted_index)

2660662

In [14]:
!head {collection_path}

0	The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.
1	The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science.
2	Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade.
3	The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to the period of the project from 194 â¦ 2-1946 under the control of the U.S. Army Corps of Engineers

## Avaliação

In [15]:
topics_file = main_path + '/pyserini/tools/topics-and-qrels/topics.dl20.txt'
qrels_eval = main_path + '/pyserini/tools/topics-and-qrels/qrels.dl20-passage.txt'

In [16]:
!head {topics_file}

1030303	who is aziz hashim
1037496	who is rep scalise?
1043135	who killed nicholas ii of russia
1045109	who owns barnhart crane
1049519	who said no one can make you feel inferior
1051399	who sings monk theme song
1056416	who was the highest career passer  rating in the nfl
1064670	why do hunters pattern their shotguns?
1065636	why do some places on my scalp feel sore
1071750	why is pete rose banned from hall of fame


In [17]:
!head {qrels_eval}

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


A seguir, a rotina de busca.  Para escore de similaridade dentre queries e documentos, foi utilizado o produto escalar. A normalização (similaridade de cossenos) não foi realizada para evitar o overhead de memória associado à construçãod e vetores do mesmo tamanho do vocabulárop (que term cerca de 2,6 milhões de termos).

In [18]:
import math
import torch.nn.functional as F

def search(query):
  doc_scores = defaultdict(int) # int (doc_id) -> int (score)
  query_tokens = preprocess_and_tokenize(query)
  n_query_tokens = len(query_tokens)
  query_counter = Counter(query_tokens)

  for token in query_counter.keys():
    if token in inverted_index:
      #Calcula TF-IDF para par (termo, query)
      query_tf = query_counter[token]/n_query_tokens
      doc_ids = inverted_index[token]["docs"]
      n_docs_contain_term = len(set(doc_ids))
      idf = math.log(n_documents / n_docs_contain_term)
      query_tf_idf = query_tf * idf

      for i, doc_id in enumerate(doc_ids):
        #Calcula TF-IDF para par (termo, documento)
        doc_tf = inverted_index[token]["tf"][i]
        doc_tf_idf = doc_tf * idf
        
        #Incrementa para implementar o produto escalar entre o "vetor" da 
        #query e o "vetor" do documento
        doc_scores[doc_id] += query_tf_idf * doc_tf_idf
    
          
  return doc_scores

In [19]:
results = search('who is aziz hashim')

In [ ]:
len(results)

245

In [20]:
def get_document_by_id(id):
  result = None
  with open(collection_path, 'r') as f:
    for line in f:
      fields = line.strip().split('\t')
      doc_id = fields[0]
      if doc_id == id:
        result = fields[1]
        break

  return result

In [21]:
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)[:10]

In [22]:
sorted_results

[(8726436, 14.123521231345256),
 (309441, 9.833169050913053),
 (7168318, 7.111725523870475),
 (7970461, 7.111725523870475),
 (1305521, 6.807578362373994),
 (1305520, 6.704433293948597),
 (3738183, 6.693388753283498),
 (1305528, 6.507244047098867),
 (8726435, 6.011509794808267),
 (1451848, 5.689380503874754)]

In [ ]:
get_document_by_id('8726436')

'Share on LinkedInShare on FacebookShare on TwitterShare on Google+. 1  Public Profile Aziz Hashim Managing Partner at NRD Capital. 2  Public Profile Hazizul Aziz Mohamad Hashim Project Accountant/ Cost control. 3  Public Profile Aziz Hashim 4  --. Public Profile syed aziz syed hashim --.'

As próximas células executam as rotinas de preparação e cálculo das métricas segundo o dataset de teste do TREC DL 2020.

In [29]:
query_to_results = dict()
i = 0

with open(topics_file, 'r') as f:
  for line in f:
      i += 1
      fields = line.strip().split('\t')
      query_id = fields[0]
      query_text = fields[1]
      results = search(query_text)
      query_to_results[int(query_id)] = sorted(results.items(), key=lambda x: x[1], reverse=True)[:10]

with open('run.dl20.tfidf.trec', 'w') as f:
  for query_id, results in query_to_results.items():
    for i, (doc_id, score) in enumerate(results):
      f.write(f'{query_id}\tQ0\t{doc_id}\t{i+1}\t{score}\ttf_idf\n')

In [24]:
!head run.dl20.boolean.trec

1030303	Q0	8726436	1	14.123521231345256	boolean
1030303	Q0	309441	2	9.833169050913053	boolean
1030303	Q0	7168318	3	7.111725523870475	boolean
1030303	Q0	7970461	4	7.111725523870475	boolean
1030303	Q0	1305521	5	6.807578362373994	boolean
1030303	Q0	1305520	6	6.704433293948597	boolean
1030303	Q0	3738183	7	6.693388753283498	boolean
1030303	Q0	1305528	8	6.507244047098867	boolean
1030303	Q0	8726435	9	6.011509794808267	boolean
1030303	Q0	1451848	10	5.689380503874754	boolean


In [25]:
!python {main_path}/pyserini/tools/scripts/msmarco/convert_msmarco_to_trec_qrels.py \
   --input {qrels_eval} \
   --output qrels.dl20.trec

Done!


In [26]:
!head qrels.dl20.trec

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


In [27]:
!chmod 755 {main_path}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval

In [30]:
!{main_path}/pyserini/tools/eval/trec_eval.9.0.4/trec_eval -c -m map -m ndcg_cut.10 -l 2 \
   qrels.dl20.trec run.dl20.tfidf.trec

map                   	all	0.0781
ndcg_cut_10           	all	0.2043
